<a href="https://colab.research.google.com/github/ZeyzilanKrkmz/ZeyzilanKrkmz/blob/main/ayinFazlarinaGoreDuyguTahminSist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import pandas as pd

# Örnek veri yükleme
# duygular.csv --> (timestamp, text, sentiment(duygu label), ay_fazi)
data = pd.read_csv("C:\\Users\\zzk60\\OneDrive\\Masaüstü\\duygular.csv")

# 1.1 Numpy kullanımı
timestamps = np.array(data['timestamp'])
sentiments = np.array(data['sentiment'])
moon_phases = np.array(data['ay_fazi'])

# 1.2 Eksik verileri doldurma
data.fillna(method='ffill', inplace=True)

# 1.3 Duygu skorlarının skaler analizi
sentiments_score = np.where(data['sentiment'] == 'positive', 1,
                           np.where(data['sentiment'] == 'negative', -1, 0))
data['sentiment_score'] = sentiments_score

# Ayın fazını one-hot encode yapma
one_hot_moon_phases = pd.get_dummies(data['ay_fazi'], prefix='phase')
data = pd.concat([data, one_hot_moon_phases], axis=1)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\zzk60\\OneDrive\\Masaüstü\\duygular.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import matplotlib.pyplot as plt

# Zaman serisi sentiment değişimini çizme
plt.figure(figsize=(12, 6))
plt.plot(data['timestamp'], data['sentiment_score'], label="Duygu Skoru", color='blue')
plt.title("Ay Fazlarına Göre Duygusal Değişim")
plt.xlabel("Zaman")
plt.ylabel("Duygu Skoru")
plt.legend()
plt.grid()
plt.show()

# Ayın fazlarının etkisini yan yana karşılaştırma
plt.figure(figsize=(10, 5))
moon_phase_avg = data.groupby('ay_fazi')['sentiment_score'].mean()
moon_phase_avg.plot(kind='bar', color=['blue', 'orange', 'green', 'red'])
plt.title("Ay Fazına Göre Ortalama Duygusal Skor")
plt.show()

NameError: name 'data' is not defined

<Figure size 1200x600 with 0 Axes>

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

# Model için veri hazırlığı
scaler = MinMaxScaler(feature_range=(0, 1))
sentiment_scaled = scaler.fit_transform(data['sentiment_score'].values.reshape(-1, 1))

# Zaman serisi veri oluşturma
time_steps = 30  # 30 günlük geçmiş veriye dayalı analiz için
X, y = [], []
for i in range(time_steps, len(sentiment_scaled)):
    X.append(sentiment_scaled[i-time_steps:i, 0])
    y.append(sentiment_scaled[i, 0])
X, y = np.array(X), np.array(y)

# LSTM Modeli
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])
model.compile(optimizer=Adam(learning_rate=0.001), loss="mean_squared_error")

# Eğitim
X = X.reshape((X.shape[0], X.shape[1], 1))
history = model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Tahmin
predictions = model.predict(X)
predictions = scaler.inverse_transform(predictions)

# Tahmini görselleştirme
plt.figure(figsize=(12, 6))
plt.plot(timestamps[time_steps:], data['sentiment_score'][time_steps:], label="Gerçek")
plt.plot(timestamps[time_steps:], predictions, label="Tahmin", linestyle="--", color="red")
plt.title("Duygusal Eğilim Tahmini")
plt.xlabel("Zaman")
plt.ylabel("Duygu Skoru")
plt.legend()
plt.show()

NameError: name 'data' is not defined